# api

This is the primary interface to running squ wrappers 

In [ ]:
#| default_exp api

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from squ.core import *
from diskcache import memoize_stampede
from concurrent.futures import ThreadPoolExecutor
import pandas, json, logging

In [ ]:
#| export
logger = logging.basicConfig(level=logging.INFO)

## List Workspaces

The `list_workspaces` function retreives a list of workspaces from blob storage and returns it in various formats

In [ ]:
#| exports
@memoize_stampede(cache, expire=60 * 60 * 3) # cache for 3 hours
def list_workspaces(fmt: str = "df", # df, csv, json, list
                    agency: str = "ALL"): # Agency alias or ALL
    path = datalake_path()
    df = pandas.read_csv((path / "notebooks/lists/SentinelWorkspaces.csv").open())
    df = df.join(pandas.read_csv((path / "notebooks/lists/SecOps Groups.csv").open()).set_index("Alias"), on="SecOps Group", rsuffix="_secops")
    df = df.rename(columns={"SecOps Group": "alias", "Domains and IPs": "domains"})
    df = df.dropna(subset=["customerId"]).sort_values(by="alias")
    if agency != "ALL":
        df = df[df["alias"] == agency]
    if fmt == "df":
        return df
    elif fmt == "csv":
        return df.to_csv()
    elif fmt == "json":
        return df.fillna("").to_dict("records")
    elif fmt == "list":
        return list(df["customerId"].unique())
    else:
        raise ValueError("Invalid format")

In [10]:
list_workspaces().head()

,DIRECTORY,ITSA Email,Information Classification,JiraOrgId,LOCATION,M365 Security Portal,RESOURCE GROUP,alias,SecOps Status,Subscription Name,...,Email Identities,Full Time Employee,ITSA Email_secops,JiraOrgId_secops,Primary Agency,Primary Agency Type,Risk Profile,SOC Intent,Status,Target SLA
39,Aqwest,it_alerts@aqwest.com.au,OFFICIAL,36.0,Australia Central,NaN,rg_aqwestcorp_azureau_central,Aqwest,Connected:T0; Risk:High - Critical Infrastructure,Aqwest Bunbury Water Corp Azure,...,NaN,42.0,it_alerts@aqwest.com.au,36.0,Aqwest,Schedule 1 Entity,High - Critical Infrastructure,Connected,Connected:T0; Risk:High - Critical Infrastructure,NaN
47,Construction Training Fund,emoore@bcitf.org,OFFICIAL,49.0,Australia East,NaN,ctf-prd-sentinel-rg,CTF,Connected:T0; Risk:Low,CTF - PROD,...,jbertram@ctf.wa.gov.au,30.0,emoore@bcitf.org,49.0,Construction Training Fund,Non-SES Entity,Low,Connected,Connected:T0; Risk:Low,SLA1
31,ChemCentre,itsa@chemcentre.wa.gov.au,OFFICIAL,32.0,Australia East,NaN,ccwa_au_ea_rg1,ChemCentre,Connected:T0; Risk:Medium,PAYG-PG_CC,...,NaN,140.0,itsa@chemcentre.wa.gov.au,32.0,Chemistry Centre (WA),SES Organisation (Schedule 2),Medium,Connected,Connected:T0; Risk:Medium,NaN
0,"Department of Biodiversity, Conservation and A...",ITSA@dbca.wa.gov.au,OFFICIAL,3.0,Australia Southeast,https://security.microsoft.com/v2/advanced-hun...,oim-appservices,DBCA,Connected:T1; Risk:Medium,OIM Azure Subscription,...,"chris.hocking@dbca.wa.gov.au, brendan.cale@dbc...",2046.0,ITSA@dbca.wa.gov.au,3.0,"Department of Biodiversity, Conservation and A...",Department (Section 35),Medium,Connected,Connected:T1; Risk:Medium,SLA1
8,Department of Fire and Emergency Services,ictsa@dfes.wa.gov.au,OFFICIAL,22.0,Australia East,NaN,azaue-sentinel-prod-01-rg,DFES,Connected:T0; Risk:High - Emergency Services,DFES-Production-EA,...,sami.anderson@dfes.wa.gov.au\nsimon.rice@dfes....,1673.0,ictsa@dfes.wa.gov.au,22.0,Department of Fire and Emergency Services,Department (Section 35),High - Emergency Services,Connected,Connected:T0; Risk:High - Emergency Services,NaN


# Log Analytics Query
The below function makes it easy to query all workspaces with sentinel installed using log analytics.

In [ ]:
#| exports
@memoize_stampede(cache, expire=60 * 60 * 3) # cache for 3 hours
def list_subscriptions():
    return pandas.DataFrame(azcli(["account", "list"]))["id"].unique()

@memoize_stampede(cache, expire=60 * 60 * 3) # cache for 3 hours
def list_securityinsights():
    return pandas.DataFrame(azcli([
        "graph", "query", "--first", "1000", "-q", 
        """
        resources
        | where type =~ 'microsoft.operationsmanagement/solutions'
        | where name startswith 'SecurityInsights'
        | project wlid = tolower(tostring(properties.workspaceResourceId))
        | join kind=leftouter (
            resources | where type =~ 'microsoft.operationalinsights/workspaces' | extend wlid = tolower(id))
            on wlid
        | extend customerId = properties.customerId
        """
    ])["data"])

def loganalytics_query(query):
    dfs = []
    customerids = list_securityinsights()["customerId"]
    with ThreadPoolExecutor(max_workers=32) as executor:
        futures = [executor.submit(azcli, [
            "monitor", "log-analytics", "query",
            "-w", workspace,
            "--analytics-query", query
        ]) for workspace in customerids]
        for future, customerid in zip(futures, customerids):
            try:
                df = pandas.DataFrame(future.result())
            except Exception as e:
                logger.warning(e)
                continue
            else:
                if "TenantId" not in df.columns:
                    df["TenantId"] = customerid
                dfs.append(df)
    return pandas.concat(dfs)

In [ ]:
loganalytics_query("SecurityIncident").head()

,AdditionalData,AlertIds,BookmarkIds,Classification,ClassificationComment,ClassificationReason,ClosedTime,Comments,CreatedTime,Description,...,RelatedAnalyticRuleIds,Severity,SourceSystem,Status,TableName,Tasks,TenantId,TimeGenerated,Title,Type
0,"{""alertsCount"":1,""bookmarksCount"":0,""commentsC...","[""af7d6f0a-2727-7681-5cdf-44cbebd8c4e0""]",[],,,,None,[],2023-04-04T06:45:06.4720273Z,Identifies when a new privileged role is assig...,...,"[""a8bc85ec-b192-4e56-b546-2fb7ec6ef21c""]",High,Azure,New,PrimaryResult,[],1f532cb7-2ac7-47c6-b892-e738ebcca160,2023-04-04T06:45:06.4720273Z,User Assigned Privileged Role,SecurityIncident
1,"{""alertsCount"":1,""bookmarksCount"":0,""commentsC...","[""92bbf5a1-cad2-55af-0e62-241308a7446e""]",[],,,,None,[],2023-04-04T06:34:49.2421114Z,This query look for applications that have bee...,...,"[""8130e4de-e7dc-4191-b1b2-8fcea65b15de""]",Medium,Azure,New,PrimaryResult,[],1f532cb7-2ac7-47c6-b892-e738ebcca160,2023-04-04T06:34:49.2421114Z,Mail.Read Permissions Granted to Application,SecurityIncident
2,"{""alertsCount"":1,""bookmarksCount"":0,""commentsC...","[""9ca05515-8576-024a-228d-bfb5dbe7705f""]",[],,,,None,[],2023-01-30T07:08:03.8585259Z,Identifies anomalous spike in frequency of exe...,...,"[""febb525a-1bd2-4cda-af79-ce89d55d2497""]",Medium,Azure,New,PrimaryResult,[],1f532cb7-2ac7-47c6-b892-e738ebcca160,2023-01-30T07:08:03.8585259Z,Process execution frequency anomaly,SecurityIncident
3,"{""alertsCount"":1,""bookmarksCount"":0,""commentsC...","[""3ead7d91-fbc5-c6c2-bb35-9375ddff0f8b""]",[],,,,None,[],2023-03-29T06:31:36.9056906Z,This will alert when an admin or app owner acc...,...,"[""4598e9e2-db7d-46ec-9a5f-8d90e61a41ea""]",High,Azure,New,PrimaryResult,[],1f532cb7-2ac7-47c6-b892-e738ebcca160,2023-03-29T06:31:36.9056906Z,First access credential added to Application o...,SecurityIncident
4,"{""alertsCount"":1,""bookmarksCount"":0,""commentsC...","[""20d3e2a8-48d7-0ef5-db2e-f8eadb9d7a4a""]",[],,,,None,[],2023-03-27T06:52:26.1842634Z,Identifies anomalous increases in Exchange mai...,...,"[""ab3e6f86-6a0d-4e0e-9eed-9dd0147a004a""]",Medium,Azure,New,PrimaryResult,[],1f532cb7-2ac7-47c6-b892-e738ebcca160,2023-03-27T06:52:26.1842634Z,Exchange workflow MailItemsAccessed operation ...,SecurityIncident


In [ ]:
from importlib.resources import path
from subprocess import run

transformer = path("squ", "atlaskit-transformer.bundle.js").absolute()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()